In [4]:
import os

In [5]:
%pwd

'd:\\mjourFinalPorject\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\mjourFinalPorject'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from src.dogvscatclassfier.constants import *
from src.dogvscatclassfier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import zipfile
import gdown
from src.dogvscatclassfier import logger
from src.dogvscatclassfier.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-01 21:45:59,677: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-01 21:45:59,679: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-01 21:45:59,680: INFO: common: created directory at: artifacts]
[2024-07-01 21:45:59,681: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-01 21:45:59,682: INFO: 3172177572: Downloading data from https://drive.google.com/file/d/1FxL7t9xGRzjkquiOGra1FI5BLS-d-Dge/view?usp=sharing into file artifacts/data_ingestion/data.rar]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1FxL7t9xGRzjkquiOGra1FI5BLS-d-Dge
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1FxL7t9xGRzjkquiOGra1FI5BLS-d-Dge&confirm=t&uuid=1e19f67d-3fcd-4243-b721-0a5457846936
To: d:\mjourFinalPorject\artifacts\data_ingestion\data.rar
100%|██████████| 812M/812M [01:46<00:00, 7.62MB/s] 


[2024-07-01 21:47:50,434: INFO: 3172177572: Downloaded data from https://drive.google.com/file/d/1FxL7t9xGRzjkquiOGra1FI5BLS-d-Dge/view?usp=sharing into file artifacts/data_ingestion/data.rar]


BadZipFile: File is not a zip file